## **Sentiment Analysis**

**Background:**

In this Notebook, we will perform the Sentiment Analysis of Twitter Tweets. The final objective iq to look at the **Network application** of the data, we won't pay a lot of attention to the results here. We just want to get our data set for the **Storytelling Network Analysis**.

**Dataset:**

We will start from the **Market Similarities Dataset** and then add Twitter Data for each Brand according to their Twitter Account. The Twitter Data contains the text of the tweets as well as additional features such as the location, the number of likes, etc.

**Resources:**

*   Pandas documentation: https://pandas.pydata.org/docs/#
*   DataCamp course: https://app.datacamp.com/learn/courses/analyzing-social-media-data-in-python
*   Data Science for Business Applications course - Copenhagen Business School
*   YouTube: https://www.youtube.com/watch?v=ujId4ipkBio&t=512s
*   Medium: https://towardsdatascience.com/nlp-part-3-exploratory-data-analysis-of-text-data-1caa8ab3f79d


## **Introduction: Libraries and Credentials** 

In [ ]:
# Import all needed libraries
import tweepy                   # Python wrapper around Twitter API
from google.colab import drive  # to mount Drive to Colab notebook

import pandas as pd
pd.set_option('display.max_colwidth', None) #to see more text
import json 
import csv
from datetime import date
from datetime import datetime
import time
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()

from textblob import TextBlob
from wordcloud import WordCloud
import string
import itertools
from collections import Counter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Connect Google Drive to Colab
drive.mount('/content/gdrive')
# Create a variable to store the data path on your drive
path = './gdrive/My Drive/path/to/data'

Mounted at /content/gdrive


In [ ]:
# Twitter API credentials

api_key = 'vpBT5zCco120S5VkWGeq0jbU3'
api_secret_key = 'nIk59NQikAS9kug5GcOkWuEqOqbAr1UgRhn5H77T9OiCdeEo7m'
access_token = '1362326919157583873-3NVcDUiiEBUsZGe06A7HzCTAs1tEDb'
access_token_secret = 'Y0Wrzi23jYd0TiAe9tNqQHax3PFqBrPRg2N2KQCl0rVtJ'

In [ ]:
# Connect to Twitter API using the secrets
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
# Helper function to save data into a JSON file
# file_name: the file name of the data on Google Drive
# file_content: the data you want to save
def save_json(file_name, file_content):
  with open(path + file_name, 'w', encoding='utf-8') as f:
    json.dump(file_content, f, ensure_ascii=False, indent=4)

In [ ]:
# Helper function to handle twitter API rate limit
def limit_handled(cursor, list_name):
  while True:
    try:
      yield cursor.next()
    # Catch Twitter API rate limit exception and wait for 15 minutes
    except tweepy.RateLimitError:
      print("\nData points in list = {}".format(len(list_name)))
      print('Hit Twitter API rate limit.')
      for i in range(3, 0, -1):
        print("Wait for {} mins.".format(i * 5))
        time.sleep(5 * 60)
    # Catch any other Twitter API exceptions
    except tweepy.error.TweepError:
      print('\nCaught TweepError exception' )

## **Get the account tweets**

First, we will import the previous dataset from the **Market Similarities Analysis**.

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Final Project/Network Analysis/Market Similarities.csv')
df = df.iloc[:,[1,2,3,13,14]]
df.head()

,Brand,Brand Owner,Twitter,Partition,Centrality
0,Cartier,Richemont SA,Cartier,0,0.538462
1,Van Cleef & Arpels,Richemont SA,vancleefarpels,1,0.615385
2,Burberry,Richemont SA,Burberry,1,0.487179
3,Baume & Mercier,Richemont SA,baumeetmercier,1,0.487179
4,IWC,Richemont SA,IWC,1,0.487179


Here is the **list** we will send to the API function to get all the tweets posts.

In [ ]:
Brands = list(df['Twitter'])
Brands

['Cartier',
 'vancleefarpels',
 'Burberry',
 'baumeetmercier',
 'IWC',
 'jaegerlecoultre',
 'Piaget',
 'RalphLauren',
 'Roger_Dubuis',
 'Vacheron1755',
 'montblanc_world',
 'alfreddunhill',
 'chloefashion',
 'petermillar',
 'Gucci',
 'YSL',
 'McQueen',
 'BALENCIAGA',
 'ulysse_nardin',
 'Boucheron',
 'GirardPerregaux',
 'jrwatches',
 'sergiorossi',
 'LouisVuitton',
 'Dior',
 'LoeweOfficial',
 'kenzo',
 'givenchy',
 'marcjacobs',
 'Fendi',
 'EmilioPucci',
 'NKirkwoodLondon',
 'Guerlain',
 'BenefitBeauty',
 'Makeupforever',
 'TAGHeuer',
 'ZenithWatches',
 'Hublot',
 'Chaumet',
 'Bulgariofficial']

**The program below will retrieve the maximum amount of tweet posts for each brand thanks to Twitter API**

In [ ]:
data = []

for Brand in Brands:
  # initialize a list to hold all the Tweets
  alltweets = []
  # make initial request for most recent tweets 
  # (200 is the maximum allowed count)
  new_tweets = api.user_timeline(Brand,count=200)
  # save most recent tweets
  alltweets.extend(new_tweets)
  # save the id of the oldest tweet less one to avoid duplication
  oldest = alltweets[-1].id - 1
  # keep grabbing tweets until there are no tweets left
  while len(new_tweets) > 0:
      print("getting tweets before %s" % (oldest))
      # all subsequent requests use the max_id param to prevent
      # duplicates
      new_tweets = api.user_timeline(Brand,count=200,max_id=oldest)
      # save most recent tweets
      alltweets.extend(new_tweets)
      # update the id of the oldest tweet less one
      oldest = alltweets[-1].id - 1
      print("...%s tweets downloaded so far" % (len(alltweets)))
      ### END OF WHILE LOOP ###

  # transform the tweepy tweets into a list 
  # populate the list
  outtweets = [[Brand, tweet.id_str, tweet.created_at, tweet.text, tweet.favorite_count,tweet.in_reply_to_screen_name, tweet.retweeted] for tweet in alltweets]
  data = data + outtweets
  dftweets = pd.DataFrame(data, columns=["brand","id","created_at","text","likes","in reply to","retweeted"])
  print(dftweets["brand"].value_counts())

getting tweets before 1412441240122138627
...400 tweets downloaded so far
getting tweets before 1365576674172166146
...600 tweets downloaded so far
getting tweets before 1275885767064199167
...800 tweets downloaded so far
getting tweets before 1184016092827664383
...1000 tweets downloaded so far
getting tweets before 1092336559565750271
...1200 tweets downloaded so far
getting tweets before 989504953935908863
...1400 tweets downloaded so far
getting tweets before 921400482651832320
...1600 tweets downloaded so far
getting tweets before 865977593853620223
...1800 tweets downloaded so far
getting tweets before 819212291958636543
...2000 tweets downloaded so far
getting tweets before 761977776366882817
...2200 tweets downloaded so far
getting tweets before 703258914784092160
...2400 tweets downloaded so far
getting tweets before 641268058095415295
...2600 tweets downloaded so far
getting tweets before 575282862469087231
...2800 tweets downloaded so far
getting tweets before 49229245189876

**Now let's have a look at our dataset!**

In [ ]:
# Check the head of the dftweets
dftweets

,brand,id,created_at,text,likes,in reply to,retweeted
0,Cartier,1458479709046648839,2021-11-10 17:00:34,Immerse yourself in the studio of Damien Hirst with this 360° experience of his work on the “Cherry Blossoms” serie… https://t.co/C5R0nD0WOu,36,None,False
1,Cartier,1458117376847880198,2021-11-09 17:00:47,"Cartier and Islamic Art: In Search of Modernity’, now open @madparisfr until 20th February 2022, highlights the inf… https://t.co/4SD8PihHC9",56,None,False
2,Cartier,1458045858625302537,2021-11-09 12:16:36,"@Joeanglo Thank you for your response. May I kindly advise to contact the client relations center in Israel, so the… https://t.co/aD3OmaATRS",0,Joeanglo,False
3,Cartier,1457978062381932545,2021-11-09 07:47:12,"@tomoid Dear Mr. Thomas, thank you for taking the time to contact us and sincerely regret the disappointment you ha… https://t.co/6EpStIjZdV",0,tomoid,False
4,Cartier,1457766124997783556,2021-11-08 17:45:02,"Un voyage depuis les origines sacrées du #parfum, de l’Egypte Antique à Rome en passant par l’Arabie, jusqu’à ses u… https://t.co/czQr0qaShY",26,Cartier,False
...,...,...,...,...,...,...,...
106370,Bulgariofficial,22829316765,2010-09-02 20:57:41,Check out Jessica Alba wearing #Bulgari at the Uomo Vogue Dinner during the Venice Film Festival: http://ow.ly/i/3zBP,0,None,False
106371,Bulgariofficial,22732640878,2010-09-01 19:00:18,@hautelivingmag loves the #Bulgari Skincare spa experience in Dallas: http://ow.ly/2xzd9,0,HauteLivingMag,False
106372,Bulgariofficial,22641123692,2010-08-31 19:00:21,@rzrachelzoe and @MRBRADGORESKI stay at the #Bulgari Hotel while taking on Milan Fashion Week. Check it out tonight at 10PM on Bravo!,0,RachelZoe,False
106373,Bulgariofficial,22523955102,2010-08-30 13:28:47,Gugu Mbatha-Raw dazzles at the Emmys in a #Bulgari platinum/diamond bracelet and pink quartz/diamond white gold earrings http://ow.ly/i/3vrR,0,None,False




> **More than 100000 rows of Tweets** to analyze!



In [ ]:
dftweets['brand'].value_counts()

Gucci              3250
IWC                3250
montblanc_world    3250
Dior               3250
McQueen            3250
TAGHeuer           3250
LouisVuitton       3250
Fendi              3250
Hublot             3249
marcjacobs         3249
Piaget             3248
baumeetmercier     3247
Guerlain           3245
LoeweOfficial      3244
jaegerlecoultre    3243
petermillar        3242
BenefitBeauty      3241
Bulgariofficial    3240
Burberry           3237
Vacheron1755       3226
RalphLauren        3225
vancleefarpels     3222
GirardPerregaux    3207
sergiorossi        3199
Makeupforever      3168
jrwatches          3139
ulysse_nardin      2917
Cartier            2914
Roger_Dubuis       1967
ZenithWatches      1950
Boucheron          1927
NKirkwoodLondon    1904
alfreddunhill      1825
YSL                1792
EmilioPucci        1675
givenchy           1518
Chaumet            1043
kenzo               737
chloefashion        121
BALENCIAGA           14
Name: brand, dtype: int64

Now let's **clean** our text column:

In [ ]:
# Clean the text

# Create a function to clean the tweets

def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9_]+', '', text) #Revoming @mentions
  text = re.sub(r'#', '', text) # Removing the '#' simbol
  text = re.sub(r'RT[\s]+', '', text) # Removing RT
  text = re.sub(r'https?:\/\/\S+', '', text) # Removing the hyper link
  text = text.lower() # make text lowercase
  text = re.sub('\[.*?\]', '', text) # removing text within brackets
  text = re.sub('\(.*?\)', '', text) # removing text within parentheses
  text = re.sub('\w*\d\w*', '', text) # removing numbers
  text = re.sub('\s+', ' ', text) # if there's more than 1 whitespace, then make it just 1
  text = re.sub('\n', ' ', text) # if there's a new line, then make it a whitespace
  text = re.sub('\"+', '', text) # removing any quotes
  text = re.sub('(\&amp\;)', '', text) # removing &amp;
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Get rid of all punctuation
  text = re.sub('(httptco)', '', text) # getting rid of `httptco`
  text = re.sub(r'[^\w\s]', '',text) # remove other punctuation

  return text

# Cleaning the text
dftweets['text'] = dftweets['text'].apply(cleanTxt)

# Show the cleaned text
dftweets.head()

,brand,id,created_at,text,likes,in reply to,retweeted
0,Cartier,1458479709046648839,2021-11-10 17:00:34,immerse yourself in the studio of damien hirst with this experience of his work on the cherry blossoms serie,36,None,False
1,Cartier,1458117376847880198,2021-11-09 17:00:47,cartier and islamic art in search of modernity now open until february highlights the inf,56,None,False
2,Cartier,1458045858625302537,2021-11-09 12:16:36,thank you for your response may i kindly advise to contact the client relations center in israel so the,0,Joeanglo,False
3,Cartier,1457978062381932545,2021-11-09 07:47:12,dear mr thomas thank you for taking the time to contact us and sincerely regret the disappointment you ha,0,tomoid,False
4,Cartier,1457766124997783556,2021-11-08 17:45:02,un voyage depuis les origines sacrées du parfum de legypte antique à rome en passant par larabie jusquà ses u,26,Cartier,False


## **Sentiment Analysis**

### **Number of words and text length**

In [ ]:
dftweets['text_len'] = dftweets['text'].apply(lambda x: len(str(x).split()))
dftweets

,brand,id,created_at,text,likes,in reply to,retweeted,Subjectivity,Polarity,text_len
0,Cartier,1458479709046648839,2021-11-10 17:00:34,immerse yourself in the studio of damien hirst with this experience of his work on the cherry blossoms serie,36,None,False,0.000000,0.00,19
1,Cartier,1458117376847880198,2021-11-09 17:00:47,cartier and islamic art in search of modernity now open until february highlights the inf,56,None,False,0.500000,0.00,15
2,Cartier,1458045858625302537,2021-11-09 12:16:36,thank you for your response may i kindly advise to contact the client relations center in israel so the,0,Joeanglo,False,0.500000,0.25,19
3,Cartier,1457978062381932545,2021-11-09 07:47:12,dear mr thomas thank you for taking the time to contact us and sincerely regret the disappointment you ha,0,tomoid,False,0.450000,-0.05,19
4,Cartier,1457766124997783556,2021-11-08 17:45:02,un voyage depuis les origines sacrées du parfum de legypte antique à rome en passant par larabie jusquà ses u,26,Cartier,False,0.000000,0.00,20
...,...,...,...,...,...,...,...,...,...,...
106370,Bulgariofficial,22829316765,2010-09-02 20:57:41,check out jessica alba wearing bulgari at the uomo vogue dinner during the venice film festival,0,None,False,0.000000,0.00,16
106371,Bulgariofficial,22732640878,2010-09-01 19:00:18,loves the bulgari skincare spa experience in dallas,0,HauteLivingMag,False,0.000000,0.00,8
106372,Bulgariofficial,22641123692,2010-08-31 19:00:21,and stay at the bulgari hotel while taking on milan fashion week check it out tonight at on bravo,0,RachelZoe,False,0.000000,0.00,19
106373,Bulgariofficial,22523955102,2010-08-30 13:28:47,gugu mbatharaw dazzles at the emmys in a bulgari platinumdiamond bracelet and pink quartzdiamond white gold earrings,0,None,False,0.150000,-0.05,17


In [ ]:
# Set number of words
dftweets.rename(columns={"text_len": "nb_words"}, inplace=True)
dftweets.head()

,brand,id,created_at,text,likes,in reply to,retweeted,Subjectivity,Polarity,nb_words
0,Cartier,1458479709046648839,2021-11-10 17:00:34,immerse yourself in the studio of damien hirst with this experience of his work on the cherry blossoms serie,36,None,False,0.00,0.00,19
1,Cartier,1458117376847880198,2021-11-09 17:00:47,cartier and islamic art in search of modernity now open until february highlights the inf,56,None,False,0.50,0.00,15
2,Cartier,1458045858625302537,2021-11-09 12:16:36,thank you for your response may i kindly advise to contact the client relations center in israel so the,0,Joeanglo,False,0.50,0.25,19
3,Cartier,1457978062381932545,2021-11-09 07:47:12,dear mr thomas thank you for taking the time to contact us and sincerely regret the disappointment you ha,0,tomoid,False,0.45,-0.05,19
4,Cartier,1457766124997783556,2021-11-08 17:45:02,un voyage depuis les origines sacrées du parfum de legypte antique à rome en passant par larabie jusquà ses u,26,Cartier,False,0.00,0.00,20


In [ ]:
# Set Text len
dftweets['text_len'] = dftweets['text'].astype(str).apply(len)
dftweets.head()

,brand,id,created_at,text,likes,in reply to,retweeted,Subjectivity,Polarity,nb_words,text_len
0,Cartier,1458479709046648839,2021-11-10 17:00:34,immerse yourself in the studio of damien hirst with this experience of his work on the cherry blossoms serie,36,None,False,0.00,0.00,19,110
1,Cartier,1458117376847880198,2021-11-09 17:00:47,cartier and islamic art in search of modernity now open until february highlights the inf,56,None,False,0.50,0.00,15,91
2,Cartier,1458045858625302537,2021-11-09 12:16:36,thank you for your response may i kindly advise to contact the client relations center in israel so the,0,Joeanglo,False,0.50,0.25,19,105
3,Cartier,1457978062381932545,2021-11-09 07:47:12,dear mr thomas thank you for taking the time to contact us and sincerely regret the disappointment you ha,0,tomoid,False,0.45,-0.05,19,107
4,Cartier,1457766124997783556,2021-11-08 17:45:02,un voyage depuis les origines sacrées du parfum de legypte antique à rome en passant par larabie jusquà ses u,26,Cartier,False,0.00,0.00,20,110


### **Subjectivity and Polarity with TextBlob**

In [ ]:
# Create a function to get the subjectivity
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

# Create two new columns
dftweets['Subjectivity'] = dftweets['text'].apply(getSubjectivity)
dftweets['Polarity'] = dftweets['text'].apply(getPolarity)

# Show the new dataframe with the new columns
dftweets.head()

,brand,id,created_at,text,likes,in reply to,retweeted,Subjectivity,Polarity
0,Cartier,1458479709046648839,2021-11-10 17:00:34,immerse yourself in the studio of damien hirst with this experience of his work on the cherry blossoms serie,36,None,False,0.00,0.00
1,Cartier,1458117376847880198,2021-11-09 17:00:47,cartier and islamic art in search of modernity now open until february highlights the inf,56,None,False,0.50,0.00
2,Cartier,1458045858625302537,2021-11-09 12:16:36,thank you for your response may i kindly advise to contact the client relations center in israel so the,0,Joeanglo,False,0.50,0.25
3,Cartier,1457978062381932545,2021-11-09 07:47:12,dear mr thomas thank you for taking the time to contact us and sincerely regret the disappointment you ha,0,tomoid,False,0.45,-0.05
4,Cartier,1457766124997783556,2021-11-08 17:45:02,un voyage depuis les origines sacrées du parfum de legypte antique à rome en passant par larabie jusquà ses u,26,Cartier,False,0.00,0.00


In [ ]:
dftweets.head()

,Unnamed: 0,brand,id,created_at,text,likes,in reply to,retweeted,Subjectivity,Polarity,nb_words,text_len
0,0,Cartier,1458479709046648839,2021-11-10 17:00:34,immerse yourself in the studio of damien hirst with this experience of his work on the cherry blossoms serie,36,NaN,False,0.00,0.00,19,110
1,1,Cartier,1458117376847880198,2021-11-09 17:00:47,cartier and islamic art in search of modernity now open until february highlights the inf,56,NaN,False,0.50,0.00,15,91
2,2,Cartier,1458045858625302537,2021-11-09 12:16:36,thank you for your response may i kindly advise to contact the client relations center in israel so the,0,Joeanglo,False,0.50,0.25,19,105
3,3,Cartier,1457978062381932545,2021-11-09 07:47:12,dear mr thomas thank you for taking the time to contact us and sincerely regret the disappointment you ha,0,tomoid,False,0.45,-0.05,19,107
4,4,Cartier,1457766124997783556,2021-11-08 17:45:02,un voyage depuis les origines sacrées du parfum de legypte antique à rome en passant par larabie jusquà ses u,26,Cartier,False,0.00,0.00,20,110


### **Polarity and Intensity with VADER**

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
sentiment = dftweets['text'].apply(lambda x: analyzer.polarity_scores(str(x)))
dftweets = pd.concat([dftweets,sentiment.apply(pd.Series)],1)

In [ ]:
dftweets.rename(columns={'Subjectivity':'subjectivity', 'Polarity':'polarity', 'neg':'negative',
                         'neu':'neutral', 'pos':'positive'}, inplace=True)
dftweets.head()

,Unnamed: 0,brand,id,created_at,text,likes,in reply to,retweeted,subjectivity,polarity,nb_words,text_len,negative,neutral,positive,compound
0,0,Cartier,1458479709046648839,2021-11-10 17:00:34,immerse yourself in the studio of damien hirst with this experience of his work on the cherry blossoms serie,36,NaN,False,0.00,0.00,19,110,0.000,1.000,0.000,0.0000
1,1,Cartier,1458117376847880198,2021-11-09 17:00:47,cartier and islamic art in search of modernity now open until february highlights the inf,56,NaN,False,0.50,0.00,15,91,0.000,1.000,0.000,0.0000
2,2,Cartier,1458045858625302537,2021-11-09 12:16:36,thank you for your response may i kindly advise to contact the client relations center in israel so the,0,Joeanglo,False,0.50,0.25,19,105,0.000,0.737,0.263,0.6908
3,3,Cartier,1457978062381932545,2021-11-09 07:47:12,dear mr thomas thank you for taking the time to contact us and sincerely regret the disappointment you ha,0,tomoid,False,0.45,-0.05,19,107,0.205,0.438,0.357,0.5423
4,4,Cartier,1457766124997783556,2021-11-08 17:45:02,un voyage depuis les origines sacrées du parfum de legypte antique à rome en passant par larabie jusquà ses u,26,Cartier,False,0.00,0.00,20,110,0.000,1.000,0.000,0.0000


### **Save CSV File**

In [ ]:
dftweets.to_csv('Tweet_Posts.csv')

In [ ]:
dftweets = pd.read_csv('/content/drive/MyDrive/Final Project/2. Sentiment Analysis/Tweet_Posts.csv')